In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
import gensim.parsing.preprocessing
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import os
# nltk.download('wordnet')

In [2]:
dir = "generated_tables"
df_video = pd.read_csv(os.path.join(dir, "video_info.csv"))
df_video["text"] = df_video["video_title"] + "\n\n" + df_video["video_description"]
df_video = df_video.rename(columns={"video_url": "url"})
df_video_short = df_video[["url", "text"]]

## https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


In [3]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}
    return(tag_dict.get(tag, nltk.corpus.wordnet.NOUN))

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    try:
        list_sentences = nltk.sent_tokenize(text)
        list_words = []
        for sentence in list_sentences:
            list_words = list_words + [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
        return(list_words)
    except Exception:
        return([])

stemmer = PorterStemmer()

def stem_word(list_words):
    try:
        list_stems = [stemmer.stem(word) for word in list_words]
        return(list_stems)
    except Exception:
        return([])

def pre_process(list_words):
    try:
        list_words = [word for word in list_words if(len(word) > 2 and word not in gensim.parsing.preprocessing.STOPWORDS)]
        return(list_words)
    except Exception:
        return([])

# Generate lemmatized words
df_video_short["words"] = df_video_short.text.apply(lemmatize_text)
df_video_short.words = df_video_short.words.apply(stem_word)
df_video_short.words = df_video_short.words.apply(pre_process)
df_video_short.head()

<ipython-input-3-ffbc4e8d6868>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_video_short["words"] = df_video_short.text.apply(lemmatize_text)
/usr/lib/python3/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,url,text,words
0,https://youtube.com/watch?v=LEUaxC_QqWE,#vAustinL Vape Trick Tutorial - How to Vape O...,"[vaustinl, vape, trick, tutori, vape, ring, ne..."
1,https://youtube.com/watch?v=7NDfG7Pdzhg,$100 Vape vs $5 Vape\n\nExpensive box mod vs a...,"[100, vape, vape, expens, box, mod, regular, d..."
2,https://youtube.com/watch?v=wTZL0xYyaBs,New Studio Vape Trick Session\n\nCatch me live...,"[new, studio, vape, trick, session, catch, liv..."
3,https://youtube.com/watch?v=pm_9MF7YbmA,CBDfx Terpene CBD Vape Pen Review - To Be Hone...,"[cbdfx, terpen, cbd, vape, pen, review, honest..."
4,https://youtube.com/watch?v=P0RYJIJd0V8,Best Satisfying Vape Tricks 2018 !!! Special T...,"[best, satisfi, vape, trick, 2018, special, ta..."


In [4]:
dict_words = gensim.corpora.Dictionary(df_video_short.words)
dict_words.filter_extremes(no_below=10, no_above=0.5)

In [5]:
list_bow_corpus = [dict_words.doc2bow(doc) for doc in df_video_short.words]

In [6]:
model_tfidf = gensim.models.TfidfModel(list_bow_corpus)
corpus_tfidf = model_tfidf[list_bow_corpus]

In [7]:
model_lda = gensim.models.LdaMulticore(list_bow_corpus, num_topics=10, id2word=dict_words, passes=2)
for idx, topic in model_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.042*"video" + 0.034*"quit" + 0.028*"smoke" + 0.028*"thi" + 0.026*"nicotin" + 0.018*"cigarett" + 0.016*"e-cigarett" + 0.013*"caus" + 0.011*"work" + 0.010*"addict"
Topic: 1 
Words: 0.050*"smoke" + 0.030*"video" + 0.028*"thi" + 0.022*"trick" + 0.021*"nicotin" + 0.020*"use" + 0.015*"like" + 0.014*"quit" + 0.013*"pen" + 0.013*"best"
Topic: 2 
Words: 0.048*"e-cigarett" + 0.035*"cigarett" + 0.034*"smoke" + 0.028*"use" + 0.021*"thi" + 0.016*"new" + 0.014*"electron" + 0.014*"medic" + 0.013*"coil" + 0.012*"health"
Topic: 3 
Words: 0.025*"today" + 0.023*"subscrib" + 0.021*"instagram" + 0.020*"twitter" + 0.020*"facebook" + 0.018*"smoke" + 0.018*"cbd" + 0.018*"use" + 0.016*"follow" + 0.015*"trick"
Topic: 4 
Words: 0.046*"video" + 0.031*"new" + 0.027*"e-cigarett" + 0.020*"visit" + 0.017*"page" + 0.017*"facebook" + 0.017*"subscrib" + 0.015*"follow" + 0.014*"medic" + 0.014*"twitter"
Topic: 5 
Words: 0.062*"review" + 0.051*"pen" + 0.029*"thi" + 0.025*"video" + 0.018*"best" + 0.017*"d

In [9]:
model_lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dict_words, passes=2)
for idx, topic in model_lda_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"trick" + 0.012*"click" + 0.011*"compil" + 0.010*"tutori" + 0.010*"guy" + 0.010*"color" + 0.010*"page" + 0.009*"video" + 0.009*"enjoy" + 0.008*"comment"
Topic: 1 
Words: 0.022*"pen" + 0.017*"batteri" + 0.016*"review" + 0.010*"electron" + 0.010*"thi" + 0.010*"code" + 0.009*"mod" + 0.009*"cartridg" + 0.009*"video" + 0.008*"cigarett"
Topic: 2 
Words: 0.020*"cigarett" + 0.014*"electron" + 0.012*"smoke" + 0.011*"nicotin" + 0.011*"e-cigarett" + 0.010*"review" + 0.010*"new" + 0.009*"morn" + 0.009*"devic" + 0.009*"juul"
Topic: 3 
Words: 0.021*"e-cigarett" + 0.013*"review" + 0.013*"today" + 0.010*"teen" + 0.009*"use" + 0.009*"expert" + 0.009*"thi" + 0.008*"video" + 0.008*"public" + 0.008*"health"
Topic: 4 
Words: 0.017*"smoke" + 0.013*"trick" + 0.012*"level" + 0.010*"let" + 0.009*"group" + 0.009*"know" + 0.009*"e-cigarett" + 0.008*"video" + 0.008*"look" + 0.008*"anoth"
Topic: 5 
Words: 0.021*"trick" + 0.012*"vapetrick" + 0.011*"beginn" + 0.011*"pod" + 0.011*"review" + 0.0